<a href="https://colab.research.google.com/github/rahiakela/coursera-deep-learning-specialization/blob/course-2-improving-deep-neural-networks/week_1_gradient_checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Checking

Welcome to this week's third programming assignment! You will be implementing gradient checking to make sure that your backpropagation implementation is correct. By completing this assignment you will:

- Implement gradient checking from scratch.

- Understand how to use the difference formula to check your backpropagation implementation.

- Recognize that your backpropagation algorithm should give you similar results as the ones you got by computing the difference formula.

- Learn how to identify which parameter's gradient was computed incorrectly.

Take your time to complete this assignment, and make sure you get the expected outputs when working through the different exercises.

This is the final assignment for this week! In this assignment you will learn to implement and use gradient checking. 

You are part of a team working to make mobile payments available globally, and are asked to build a deep learning model to detect fraud--whenever someone makes a payment, you want to see if the payment might be fraudulent, such as if the user's account has been taken over by a hacker. 

But backpropagation is quite challenging to implement, and sometimes has bugs. Because this is a mission-critical application, your company's CEO wants to be really certain that your implementation of backpropagation is correct. Your CEO says, "Give me a proof that your backpropagation is actually working!" To give this reassurance, you are going to use "gradient checking".

Let's do it!

In [0]:
# Packages
import numpy as np
from testCases import *
from gc_utils import sigmoid, relu, dictionary_to_vector, vector_to_dictionary, gradients_to_vector

## 1) How does gradient checking work?

Backpropagation computes the gradients $\frac{\partial J}{\partial \theta}$, where $\theta$ denotes the parameters of the model. $J$ is computed using forward propagation and your loss function.

Because forward propagation is relatively easy to implement, you're confident you got that right, and so you're almost  100% sure that you're computing the cost $J$ correctly. Thus, you can use your code for computing $J$ to verify the code for computing $\frac{\partial J}{\partial \theta}$. 

Let's look back at the definition of a derivative (or gradient):
$$ \frac{\partial J}{\partial \theta} = \lim_{\varepsilon \to 0} \frac{J(\theta + \varepsilon) - J(\theta - \varepsilon)}{2 \varepsilon} \tag{1}$$

If you're not familiar with the "$\displaystyle \lim_{\varepsilon \to 0}$" notation, it's just a way of saying "when $\varepsilon$ is really really small."

We know the following:

- $\frac{\partial J}{\partial \theta}$ is what you want to make sure you're computing correctly. 
- You can compute $J(\theta + \varepsilon)$ and $J(\theta - \varepsilon)$ (in the case that $\theta$ is a real number), since you're confident your implementation for $J$ is correct. 

Lets use equation (1) and a small value for $\varepsilon$ to convince your CEO that your code for computing  $\frac{\partial J}{\partial \theta}$ is correct!

<img src='https://github.com/rahiakela/img-repo/blob/master/deep-learning-specialization/gradient-checking-derivative-computation.png?raw=1' width='800'/>

## 2) 1-dimensional gradient checking

Consider a 1D linear function $J(\theta) = \theta x$. The model contains only a single real-valued parameter $\theta$, and takes $x$ as input.

You will implement code to compute $J(.)$ and its derivative $\frac{\partial J}{\partial \theta}$. You will then use gradient checking to make sure your derivative computation for $J$ is correct. 

<img src='https://github.com/rahiakela/img-repo/blob/master/deep-learning-specialization/1Dgrad_kiank.png?raw=1' width='800'/>
<caption><center> <u> **Figure 1** </u>: **1D linear model**<br> </center></caption>

The diagram above shows the key computation steps: First start with $x$, then evaluate the function $J(x)$ ("forward propagation"). Then compute the derivative $\frac{\partial J}{\partial \theta}$ ("backward propagation"). 

**Exercise**: implement "forward propagation" and "backward propagation" for this simple function. I.e., compute both $J(.)$ ("forward propagation") and its derivative with respect to $\theta$ ("backward propagation"), in two separate functions.

In [0]:
# FUNCTION: forward_propagation
def forward_propagation(x, theta):
  '''
  Implement the linear forward propagation (compute J) presented in Figure 1 (J(theta) = theta * x)
    
  Arguments:
  x -- a real-valued input
  theta -- our parameter, a real number as well
  
  Returns:
  J -- the value of function J, computed using the formula J(theta) = theta * x
  '''

  ### START CODE HERE ### (approx. 1 line)
  J = np.dot(theta, x)
  ### END CODE HERE ###

  return J

In [33]:
x, theta = 2, 4
J = forward_propagation(x, theta)
print(f'J = {str(J)}')

J = 8


**Expected Output**:

<table style=>
    <tr>
        <td>  ** J **  </td>
        <td> 8</td>
    </tr>
</table>

**Exercise**: Now, implement the backward propagation step (derivative computation) of Figure 1. That is, compute the derivative of $J(\theta) = \theta x$ with respect to $\theta$. To save you from doing the calculus, you should get $dtheta = \frac { \partial J }{ \partial \theta} = x$.

In [0]:
# FUNCTION: backward_propagation
def backward_propagation(x, theta):
  '''
  Computes the derivative of J with respect to theta (see Figure 1).
    
  Arguments:
  x -- a real-valued input
  theta -- our parameter, a real number as well
  
  Returns:
  dtheta -- the gradient of the cost with respect to theta
  '''

  ### START CODE HERE ### (approx. 1 line)
  dtheta = np.dot(theta, x) / theta
  ### END CODE HERE ###

  return dtheta

In [47]:
x, theta = 2, 4
dtheta = backward_propagation(x, theta)
print(f'dtheta = {str(dtheta)}')

dtheta = 2.0


**Expected Output**:

<table>
    <tr>
        <td>  ** dtheta **  </td>
        <td> 2 </td>
    </tr>
</table>

**Exercise**: To show that the `backward_propagation()` function is correctly computing the gradient $\frac{\partial J}{\partial \theta}$, let's implement gradient checking.

**Instructions**:
- First compute "gradapprox" using the formula above (1) and a small value of $\varepsilon$. Here are the Steps to follow:
    1. $\theta^{+} = \theta + \varepsilon$
    2. $\theta^{-} = \theta - \varepsilon$
    3. $J^{+} = J(\theta^{+})$
    4. $J^{-} = J(\theta^{-})$
    5. $gradapprox = \frac{J^{+} - J^{-}}{2  \varepsilon}$
- Then compute the gradient using backward propagation, and store the result in a variable "grad"
- Finally, compute the relative difference between "gradapprox" and the "grad" using the following formula:
$$ difference = \frac {\mid\mid grad - gradapprox \mid\mid_2}{\mid\mid grad \mid\mid_2 + \mid\mid gradapprox \mid\mid_2} \tag{2}$$
You will need 3 Steps to compute this formula:
   - 1'. compute the numerator using np.linalg.norm(...)
   - 2'. compute the denominator. You will need to call np.linalg.norm(...) twice.
   - 3'. divide them.
- If this difference is small (say less than $10^{-7}$), you can be quite confident that you have computed your gradient correctly. Otherwise, there may be a mistake in the gradient computation. 


In [0]:
# FUNCTION: gradient_check
def gradient_check(x, theta, epsilon=1e-7):
  '''
  Implement the backward propagation presented in Figure 1.
    
  Arguments:
  x -- a real-valued input
  theta -- our parameter, a real number as well
  epsilon -- tiny shift to the input to compute approximated gradient with formula(1)
  
  Returns:
  difference -- difference (2) between the approximated gradient and the backward propagation gradient
  '''

  # Compute gradapprox using left side of formula (1). epsilon is small enough, you don't need to worry about the limit.
  ### START CODE HERE ### (approx. 5 lines)
  thetaplus = theta + epsilon                              # Step 1
  thetaminus = theta - epsilon                             # Step 2
  J_plus =np.dot(x, thetaplus)                             # Step 3
  J_minus = np.dot(x, thetaminus)                          # Step 4
  gradapprox = (J_plus - J_minus) / (2 * epsilon)          # Step 5
  ### END CODE HERE ###

  # Check if gradapprox is close enough to the output of backward_propagation()
  ### START CODE HERE ### (approx. 1 line)
  grad = backward_propagation(x, theta)
  ### END CODE HERE ###

  ### START CODE HERE ### (approx. 1 line)
  numerator = np.linalg.norm(grad - gradapprox)                       # Step 1'
  denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)     # Step 2'
  difference = numerator / denominator                                # Step 3'
  ### END CODE HERE ###

  if difference < 1e-7:
    print('The gradient is correct!')
  else:
    print('The gradient is wrong!')

  return difference

In [49]:
x, theta = 2, 4
difference = gradient_check(x, theta)
print(f'difference = {str(difference)}')

The gradient is correct!
difference = 2.919335883291695e-10


**Expected Output**:
The gradient is correct!
<table>
    <tr>
        <td>  ** difference **  </td>
        <td> 2.9193358103083e-10 </td>
    </tr>
</table>

Congrats, the difference is smaller than the $10^{-7}$ threshold. So you can have high confidence that you've correctly computed the gradient in `backward_propagation()`. 

Now, in the more general case, your cost function $J$ has more than a single 1D input. When you are training a neural network, $\theta$ actually consists of multiple matrices $W^{[l]}$ and biases $b^{[l]}$! It is important to know how to do a gradient check with higher-dimensional inputs. Let's do it!

## 3) N-dimensional gradient checking

The following figure describes the forward and backward propagation of your fraud detection model.

<img src='https://github.com/rahiakela/img-repo/blob/master/deep-learning-specialization/NDgrad_kiank.png?raw=1' width='800'/>
<caption><center> <u> **Figure 2** </u>: **deep neural network**<br>*LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID*</center></caption>

Let's look at your implementations for forward propagation and backward propagation. 